# Adult
## Black-box property inference adversary

In [ ]:
import pandas as pd

# load shadow model outputs
adv_df = pd.read_csv("adult/data/shadow_model_outputs_proba-new.csv")
adv_ddf_shuffled = adv_df.sample(frac=1, random_state=1).reset_index(drop=True)
adv_y = adv_ddf_shuffled["y"]
adv_X = adv_ddf_shuffled.drop(columns=["y"])

adv_df_test = pd.read_csv("adult/data/shadow_model_outputs_proba-new_test_set.csv")
adv_ddf_shuffled_test = adv_df_test.sample(frac=1, random_state=1).reset_index(drop=True)#.drop(columns=["Unnamed: 0"])
adv_y_test = adv_ddf_shuffled_test["y"]
adv_X_test = adv_ddf_shuffled_test.drop(columns=["y"])

In [1]:
import numpy as np
import keras
from adult_functions import adult_adversary

adversary = adult_adversary((adv_X.shape[1],))
adversary.load_weights('adult/models/adv_new_0.65_test_r2')

2024-01-29 13:30:25.512067: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-29 13:30:25.512125: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-29 13:30:25.541238: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-29 13:30:25.659455: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-29 13:30:26.874212: W tensorflow/compiler/tf2

In [ ]:
# train adversary
checkpoint_filepath = 'adult/models/manual_tuning_checkpoints/keras.weights.h5'
adversary = adult_adversary((adv_X.shape[1],))
currentmax = 0.5
for i in range(100): # multiple attempts
    history = adversary.fit(
        adv_X,
        adv_y,
        epochs=200,
        validation_data=(adv_X_test, adv_y_test),
        verbose=0,
        batch_size=10,
        callbacks=[
            keras.callbacks.EarlyStopping('val_r2_score', mode='max', patience=20, verbose=1),
            keras.callbacks.ModelCheckpoint(
                filepath=checkpoint_filepath,
                save_weights_only=True,
                monitor='val_r2_score',
                mode='max',
                save_best_only=True)
        ])
    newmax = max(history.history['val_r2_score'])
    if newmax > currentmax:
        print(f"new max r2: {newmax}")
        currentmax = newmax
        adversary.load_weights("models/manual_tuning_checkpoints/keras.weights.h5")
        adversary.save(f"adv_new_new_keras_{round(float(newmax),2)}_test_r2.keras")

## Create (target) model

In [2]:
from adult_functions import train_gradient_boosting_shadow_model, data_train_test
import pandas as pd

In [3]:
# load data
X_train, X_test, y_train, y_test, sensitive, sensitive_t = data_train_test()
# create model
gb = train_gradient_boosting_shadow_model(X_train, y_train, 0)

## Generate model output

In [4]:
# load synthetic model input
model_input = pd.read_csv('adult/data/syn_data-new.csv')
# generate model output
out = gb.predict_proba(model_input)
# reformat model output
adversary_input = np.expand_dims(out[:,0], axis=0)

## Attack

In [5]:
# generate adversary prediction
adversary.predict(adversary_input)

1/1 [==============================] - 0s 92ms/step


array([[0.8010835]], dtype=float32)

Result interpretation: The adversary has inferred that 80.1% of the target model's training data are male samples. 